In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
import base64
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "130411"

shelter = AnimalShelter(username, password)
df = pd.DataFrame.from_records(shelter.read_all({}))

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project Two')
 
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard Project 2'))),
    html.Hr(),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.H5('Select one type of Rescue'),
    html.Div(
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue',    'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain or Wildernerss Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset - default', 'value': 'Reset'}
            ],
            value='Reset', multi=False, clearable=False, style={"width": "50%"}
    )),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            #dcc.Graph(id='graph-id'),
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if filter_type == 'Water Rescue':
        df = pd.DataFrame.from_records(shelter.read_all(
            {'$or': [{'breed': 'Labrador Retriever Mix'}, 
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland'}],
                    'sex_upon_outcome': 'Intact Female',
                    'age_upon_outcome_in_weeks': {'$gt': 26, '$lt': 156},
            }))    

    elif filter_type == 'Mountain or Wildernerss Rescue':
        df = pd.DataFrame.from_records(shelter.read_all(
            {'$or': [{'breed': 'German Shepherd'}, {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'}, {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'}],
                    'sex_upon_outcome': 'Intact Male',
                    'age_upon_outcome_in_weeks': {'$gt': 26, '$lt': 156},
            }))  

    elif filter_type == 'Disaster or Individual Tracking':
        df = pd.DataFrame.from_records(shelter.read_all(
            {'$or': [{'breed': 'Doberman Pinscher'}, {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'}, {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}],
                    'sex_upon_outcome': 'Intact Male',
                    'age_upon_outcome_in_weeks': {'$gt': 20, '$lt': 300},
            })) 
    
    elif filter_type == 'Reset':
        df = pd.DataFrame.from_records(shelter.read_all({}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
          
    return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()

    #add code for chart of your choice (e.g. pie chart)
    return  [
        dcc.Graph(
            figure=px.pie(data_frame=dff, values=values, names=names,
                color_discrete_sequence=px.colors.sequential.RdBu,width=800,height=500)
        )
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            #Marker with tool tip and popup
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ]) 
            ])
        ])
    ]

app